In [37]:
%pylab inline
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
from IPython.display import display
import json
import copy
import CustomObjectListsToJSON
%matplotlib notebook

from time import sleep


from enum import IntEnum

class locations(IntEnum):
#locations
    Break_Room  = 0
    Conference_Room = 1 
    Annex = 2
    Michaels_Office = 3 
    Parking_Lot = 4
    Warehouse = 5
    Kitchen = 6
    Reception = 7
    Accounting = 8
    
    def has_value(value):
        if(type(value) == str):
            return False
#         print(value)
        return value <= 8 and value >= 0

#weapons
class weapons(IntEnum):
    Rabid_Bat  = 9
    Mr_A_Knife = 10 
    Worlds_Best_Boss_Mug = 11
    Poisoned_Pretzel = 12
    Dundy_Trophy = 13
    Bicycle_Chain = 14
    Nunchunks = 15
    Ream_Of_Paper = 16
    Bacon_Grill = 17
    
    def has_value(value):
        if(type(value) == str):
            return False
#         print(value)
        return value <= 17 and value >= 9
    
#people

class people(IntEnum):
    Jim  = 18
    Stanley = 19 
    Dwight = 20
    Andy = 21
    Angela = 22
    Pam = 23
    
    def has_value(value):
        if(type(value) == str):
            return False
#         print(value)
        return value <= 23 and value >= 18
    

stored_guesses_for_replay = []   


    

# locations = ["Break Room", "Conference Room", "Annex", "Michaels Office", "Parking Lot", "Warehouse",
#              "Kitchen", "Reception", "Accounting"]

# weapons = ["Rabid Bat", "Mr A Knife", "Worlds Best Boss Mug", "Poisoned Pretzel", "Dundy Trophy", "Bicycle Chan",
#           "Nunchucks", "Ream Of Paper", "Bacon Grill"]


# people = ["Jim", "Dwight", "Stanley", "Pam", "Angela", "Andy"]

number_players = 4


player1_known = []

player2_known = []

player3_known = []

player4_known = []

players_known = [player1_known, player2_known, player3_known, player4_known]



player1_cannot_have = []

player2_cannot_have = []

player3_cannot_have = []

player4_cannot_have = []

players_cannot_have = [player1_cannot_have, player2_cannot_have, player3_cannot_have, player4_cannot_have]

solutions = []

sales = None


player1_unproven_guesses = []


all_guesses_list = []

#a list to track everything for undoing a mistake when inputting data
added_list = []



def remove_most_recently_added(_):
    global added_list
    global sales
    
    
    with out:
        
        clear_output(wait=True)
        
        if(len(added_list) == 0):
            return

        list_to_remove_from = added_list[len(added_list)-1][1]

        if(list_to_remove_from == sales):
            sales = None
        else:
            list_to_remove_from.remove(added_list[len(added_list)-1][0])

        added_list = added_list[:-1]
        
        print_players()
    
def fill_p1_cannot_have(_):
    
    global player1_known
    global player1_cannot_have
    
    with out:
        clear_output(wait=True)
        
        for loc in locations:
            if loc in player1_known:
                continue
            else:
                player1_cannot_have.append(loc)
                added_list.append([loc, player1_cannot_have])
        for weap in weapons:
            if weap in player1_known:
                continue
            else:
                player1_cannot_have.append(weap)
                added_list.append([weap, player1_cannot_have])
        for pers in people:
            if pers in player1_known:
                continue
            else:
                player1_cannot_have.append(pers)
                added_list.append([pers, player1_cannot_have])
        print_players()
        
        
#if a player is known to have a card no other players can
def fill_other_players_cannot_have(item, player_index):
    
    global number_players
    global players_cannot_have
    
    if(type(item) == str):
        return
        
#     print("item", item, "player_index", player_index)
    
#     sleep(15)
    
    for i in range(number_players):
        if(i == player_index):
            continue
        else:
            if(item not in players_cannot_have[i]):
                players_cannot_have[i].append(item)
                added_list.append([item, players_cannot_have[i]])
                
                
def add_guess_to_list(location, weapon, person, guesser, prover):
    global all_guesses_list
    
    guess = [location, weapon, person, guesser, prover]
    
    all_guesses_list.append(guess)
    added_list.append([guess, all_guesses_list])


def value_in_players_cannot_except_prover(prover_index, value):
    global players_cannot_have
    global number_players

    
    for i in range(number_players):
        if(i == prover_index):
            continue
        else:
            if(value in players_cannot_have[i]):
                return False
            
    return True
    

        
def look_through_all_previous_guesses_for_answers():
    global all_guesses_list
    global players_known
    global players_cannot_have
    global sales
    global solutions
    
    for guess in all_guesses_list:
        location = guess[0]
        weapon = guess[1]
        person = guess[2]
        guesser_index = guess[3]
        prover_index = guess[4]
        
        print("THE GUESS IS", guess)
        
        
        

        
        if(prover_index != None):
            if(location in players_known[prover_index] or weapon in players_known[prover_index] or person in players_known[prover_index]):
                continue
            prover_might_have_had_location = not location in players_cannot_have[prover_index] and sales != location and location not in solutions and sales != None and not check_if_others_have(location) 
            prover_might_have_had_weapon = not weapon in players_cannot_have[prover_index] and sales != weapon and weapon not in solutions and sales != None and not check_if_others_have(weapon) 
            prover_might_have_had_person = not person in players_cannot_have[prover_index] and sales != person and person not in solutions and sales != None and not check_if_others_have(person) 
            
            if(prover_might_have_had_location and not prover_might_have_had_weapon and not prover_might_have_had_person):
                players_known[prover_index].append(location)
                print("added ", location, "to player ", prover_index + 1)
                with open('addedlocation.txt', 'w') as out_file:
                    json.dump("added " + str(location) + "to player " + str(prover_index + 1), out_file)
                added_list.append([location, players_known[prover_index]])
                fill_other_players_cannot_have(location, prover_index)
            elif(not prover_might_have_had_location and prover_might_have_had_weapon and not prover_might_have_had_person):
                players_known[prover_index].append(weapon)
                print("added ", weapon, "to player ", prover_index + 1)
                with open('addedweapon.txt', 'w') as out_file:
                    json.dump("added " + str(weapon) + "to player " + str(prover_index + 1), out_file)
                added_list.append([weapon, players_known[prover_index]])
                fill_other_players_cannot_have(weapon, prover_index)
            elif(not prover_might_have_had_location and not prover_might_have_had_weapon and prover_might_have_had_person):
                players_known[prover_index].append(person)
                print("added ", person, "to player ", prover_index + 1)
                with open('addedperson.txt', 'w') as out_file:
                    json.dump("added " + str(person) + "to player " + str(prover_index + 1), out_file)
                added_list.append([person, players_known[prover_index]])
                fill_other_players_cannot_have(person, prover_index)
        else:
            nobody_had_the_location = set(location in player for player in players_cannot_have)
            nobody_had_the_location.add(sales != location and sales != None)
            nobody_had_the_location.add(location not in solutions and not check_if_others_have(location))
            
            if(len(nobody_had_the_location) == 1 and True in nobody_had_the_location):
                solutions.append(location)
                added_list.append([location, solutions])
                print("added location", location)
                time.sleep(15)
            
            nobody_had_the_weapon = set(weapon in player for player in players_cannot_have)
            nobody_had_the_weapon.add(sales != weapon and sales != None)
            nobody_had_the_weapon.add(weapon not in solutions and not check_if_others_have(weapon))
            
            if(len(nobody_had_the_weapon) == 1 and True in nobody_had_the_weapon):
                solutions.append(weapon)
                added_list.append([weapon, solutions])
            
            nobody_had_the_person = set(person in player for player in players_cannot_have)
            nobody_had_the_person.add(sales != person and sales != None)
            nobody_had_the_person.add(person not in solutions and not check_if_others_have(person))
            
            if(len(nobody_had_the_person) == 1 and True in nobody_had_the_person):
                solutions.append(person)
                added_list.append([person, solutions])
                


layout = widgets.Layout(width='auto', height='40px') #set width and height


add_player_hand_location = widgets.Button(description='Add Location to Selected Player', layout = layout)
add_player_hand_weapon = widgets.Button(description='Add Weapon to Selected Player', layout = layout)
add_player_hand_person = widgets.Button(description='Add Person to Selected Player', layout = layout)
fill_player1_cannot_have = widgets.Button(description='Fill P1 Cannot Have', layout = layout)
add_guess = widgets.Button(description='Add Guess', layout = layout)
add_location_sales = widgets.Button(description='Set Sales Location', layout = layout)
add_weapon_sales = widgets.Button(description='Set Sales Weapon', layout = layout)
add_person_sales = widgets.Button(description='Set Sales Person', layout = layout)
undo_previous = widgets.Button(description='Undo Previous', layout = layout)
refresh = widgets.Button(description='Refresh', layout = layout)
store = widgets.Button(description='Store Guesses', layout = layout)
restore = widgets.Button(description='Restore Guesses', layout = layout)
choose_location = widgets.Dropdown(
    options=[("(None)", None),
        ("Break Room", locations.Break_Room),
("Conference Room", locations.Conference_Room), 
("Annex", locations.Annex),
("Michaels Office", locations.Michaels_Office), 
("Parking Lot", locations.Parking_Lot),
("Warehouse", locations.Warehouse),
("Kitchen", locations.Kitchen),
("Reception", locations.Reception),
("Accounting", locations.Accounting)],
    value=None,
    description='Location:',
    layout = layout,
    style={'description_width': 'initial'},
)


choose_weapon = widgets.Dropdown(
    options= [("(None)", None),
    ("Rabid Bat", weapons.Rabid_Bat),
    ("Mr A Knife", weapons.Mr_A_Knife) ,
    ("Worlds Best Boss Mug", weapons.Worlds_Best_Boss_Mug) , 
    ("Poisoned Pretzel", weapons.Poisoned_Pretzel) , 
    ("Dundy Trophy", weapons.Dundy_Trophy) , 
    ("Bicycle Chain", weapons.Bicycle_Chain) , 
    ("Nunchunks", weapons.Nunchunks), 
    ("Ream Of Paper", weapons.Ream_Of_Paper) , 
    ("Bacon Grill", weapons.Bacon_Grill)],
    value=None,
    description='Weapon:',
    layout = layout,
    style={'description_width': 'initial'},
)


choose_person = widgets.Dropdown(
    options= [("(None)", None),
    ("Jim", people.Jim),
    ("Dwight", people.Dwight) ,
    ("Pam", people.Pam) , 
    ("Andy", people.Andy) , 
    ("Angela", people.Angela) , 
    ("Stanley", people.Stanley)],
    value=None,
    description='Person:',
    layout = layout,
    style={'description_width': 'initial'},
)


choose_player = widgets.Dropdown(
    options= [("(None)", None),
    ("Player 1", 0),
    ("Player 2", 1) ,
    ("Player 3", 2) , 
    ("Player 4", 3)],
    value=None,
    description='Player To Add To:',
    layout = layout,
    style={'description_width': 'initial'},
)

choose_player_guesser = widgets.Dropdown(
    options= [("(None)", None),
    ("Player 1", 0),
    ("Player 2", 1) ,
    ("Player 3", 2) , 
    ("Player 4", 3)],
    value=None,
    description='Player Guesser:',
    layout = layout,
    style={'description_width': 'initial'},
)

choose_player_prover = widgets.Dropdown(
    options= [("(None)", None),
    ("Player 1", 0),
    ("Player 2", 1) ,
    ("Player 3", 2) , 
    ("Player 4", 3)],
    value=None,
    description='Player Prover:',
    layout = layout,
    style={'description_width': 'initial'},
)



def list_intenums_to_names(list_intenums, avoid_indices):
    
    global all_guesses_list
    
    list_intenums_copy = copy.deepcopy(list_intenums)
    
    
#     print(list_intenums)
    print(list_intenums)
    
    for i in range(len(list_intenums_copy)):
        if(i not in avoid_indices and type(list_intenums_copy[i]) != str ):
            list_intenums_copy[i] = list_intenums_copy[i].name
    
#     if(list_intenums == all_guesses_list):
#         print(list_intenums)
#         print(list_intenums_copy)
#         print("SLEEPING")
#         time.sleep(15)
    
    return list_intenums_copy



# right = widgets.Button(description='Move Right')
# down = widgets.Button(description='Move Down')
# up = widgets.Button(description='Move Up')
# left = widgets.Button(description='Move Left')
# add_good = widgets.Button(description='Add Good Data')
# add_bad = widgets.Button(description='Add Bad Data')
# predictbtn = widgets.Button(description='Predict Button')
out = widgets.Output()



def check_definite_solutions():
    
    global solutions
    
    all_locations = []
    all_locations_bools = []
    true_count = 0
    
    for loc in locations:
        someone_else_has = check_if_others_have(loc)
        if(someone_else_has):
            true_count += 1
        all_locations.append(loc)
        all_locations_bools.append(someone_else_has)
        
    if(true_count == 8):
        last_location = all_locations[all_locations_bools.index(False)]
        if(last_location not in solutions):
            solutions.append(last_location)
            
    all_weapons = []
    all_weapons_bools = []
    true_count = 0
    
    for weap in weapons:
        someone_else_has = check_if_others_have(weap)
        if(someone_else_has):
            true_count += 1
        all_weapons.append(weap)
        all_weapons_bools.append(someone_else_has)
        
    if(true_count == 8):
        last_weapon = all_weapons[all_weapons_bools.index(False)]
        if(last_weapon not in solutions):
            solutions.append(last_weapon)
            
    all_people = []
    all_people_bools = []
    true_count = 0
    
    for pers in people:
        someone_else_has = check_if_others_have(pers)
        if(someone_else_has):
            true_count += 1
        all_people.append(pers)
        all_people_bools.append(someone_else_has)
        
    if(true_count == 8):
        last_person = all_people[all_people_bools.index(False)]
        if(last_person not in solutions):
            solutions.append(last_person)
        
        
    
    
    
    
    
        
    


def print_players_refresh(_):
    global player1_known
    global player2_known
    global player3_known
    global player4_known
    
    global player1_cannot_have
    global player2_cannot_have
    global player3_cannot_have
    global player4_cannot_have
    
    global sales
    
    global solutions
    
    global player1_unproven_guesses
    
    global all_guesses_list
    
    with out:
        
        clear_output(wait=True)
        
        check_definite_solutions()
        
        
        print("Player 1 Known: ", [i.name for i in player1_known], "Player 1 Cannot Have", [i.name for i in player1_cannot_have])
        print("Player 2 Known: ", [i.name for i in player2_known], "Player 2 Cannot Have", [i.name for i in player2_cannot_have])
        print("Player 3 Known: ", [i.name for i in player3_known], "Player 3 Cannot Have", [i.name for i in player3_cannot_have])
        print("Player 4 Known: ", [i.name for i in player4_known], "Player 4 Cannot Have", [i.name for i in player4_cannot_have])

        if(sales == None):
            print("Sales = ")
        else:  
            print("Sales = ", sales.name)

        print("Solutions", list_intenums_to_names(solutions, [None]))
        
        print("Player 1 Unproven Guesses", [list_intenums_to_names(p1guess, [None]) for p1guess in player1_unproven_guesses])
        
        print("ALL GUESSES SPECIAL 1", all_guesses_list)
        print("All Guesses", [list_intenums_to_names(guess, [3,4]) for guess in all_guesses_list])
        print("ALL GUESSES SPECIAL 2")
        
        look_for_solutions_unproven_guesses()
    
    
def print_players():
    global player1_known
    global player2_known
    global player3_known
    global player4_known
    
    global player1_cannot_have
    global player2_cannot_have
    global player3_cannot_have
    global player4_cannot_have

    
    global sales
    
    global solutions
    
    global player1_unproven_guesses
    
    global all_guesses_list
    
    with out:
        
        clear_output(wait=True)
    
        print("Player 1 Known: ", [i.name for i in player1_known], "Player 1 Cannot Have", [i.name for i in player1_cannot_have])
        print("Player 2 Known: ", [i.name for i in player2_known], "Player 2 Cannot Have", [i.name for i in player2_cannot_have])
        print("Player 3 Known: ", [i.name for i in player3_known], "Player 3 Cannot Have", [i.name for i in player3_cannot_have])
        print("Player 4 Known: ", [i.name for i in player4_known], "Player 4 Cannot Have", [i.name for i in player4_cannot_have])

        if(sales == None):
            print("Sales = ")
        else:  
            print("Sales = ", sales.name)

        print("Solutions", list_intenums_to_names(solutions, [None]))
        
        print("Player 1 Unproven Guesses", [list_intenums_to_names(p1guess,[None]) for p1guess in player1_unproven_guesses])
        
        print("All Guesses", [list_intenums_to_names(guess, [3,4]) for guess in all_guesses_list])
    
def list_contains_value(my_list, val):
    for item in my_list:
        if(item == val):
            return True
    
    return False



def check_if_others_have(val):
    global player1_known
    global player2_known
    global player3_known
    global player4_known
    global solutions
    global sales
    
    if val in player1_known:
        print("error player 1 has")
        return True
    if val in player2_known:
        print("error player 2 has")
        return True
    if val in player3_known:
        print("error player 3 has")
        return True
    if val in player4_known:
        print("error player 4 has")
        return True
    if val in solutions:
        print("error solutions has")
        return True
    if(val == sales):
        print("error that is the sales val")
        return True
    
    return False

def cross_reference_cannot_have(prover_index, location, weapon, person):
    
    global players_cannot_have
    global players_known
    
    cannot_have_location = False
    cannot_have_weapon = False
    cannot_have_person = False
    
    prover_cannot_have = players_cannot_have[prover_index]
    
    if location in prover_cannot_have:
        cannot_have_location = True
    if weapon in prover_cannot_have:
        cannot_have_weapon = True
    if person in prover_cannot_have:
        cannot_have_person = True
        
    prover_known = players_known[prover_index]
    
    if( (cannot_have_location == False) and cannot_have_weapon and cannot_have_person and location not in prover_known):
        prover_known.append(location)
        added_list.append([location, prover_known])
        fill_other_players_cannot_have(location, prover_index)
    elif(cannot_have_location and (cannot_have_weapon == False) and cannot_have_person and weapon not in prover_known):
        prover_known.append(weapon)
        added_list.append([weapon, prover_known])
        fill_other_players_cannot_have(weapon, prover_index)
    elif(cannot_have_location and cannot_have_weapon and (cannot_have_person == False) and person not in prover_known):
        prover_known.append(person)
        added_list.append([person, prover_known])
        fill_other_players_cannot_have(person, prover_index)
    
    if("annex" in players_known[1]):
        with open('errorcrossref.txt', 'w') as out_file:
            json.dump("added_anex ", out_file)
            exit()
    
        
#if the sales card is known after an unproven guess
#this can prove the guess might have been important
def look_for_solutions_unproven_guesses():
    
    global player1_unproven_guesses
    global player1_known
    
    for guess in player1_unproven_guesses:
        location = guess[0]
        weapon = guess[1]
        person = guess[2]
        
        #none of the others can have them if player 1 guessed
        #and no one proved
        fill_other_players_cannot_have(location, 0)
        fill_other_players_cannot_have(weapon, 0)
        fill_other_players_cannot_have(person, 0)
        
        if(sales != None):
            if(sales != location and check_if_others_have(location) == False and location not in player1_known):
                solutions.append(location)
                added_list.append([location, solutions])
                print("added location2", location)
                time.sleep(15)
            if(sales != weapon and check_if_others_have(weapon) == False):
                solutions.append(weapon)
                added_list.append([weapon, solutions])
            if(sales != person and check_if_others_have(person) == False):
                solutions.append(person)
                added_list.append([person, solutions])
    


def check_for_definite_solutions():
    global player1_cannot_have
    global player2_cannot_have
    global player3_cannot_have
    global player4_cannot_have
    global sales
    
    if(sales != None):
        

    
        for val1 in player1_cannot_have:
            if(val1 == sales):
                continue
            for val2 in player2_cannot_have:
                if(val1 != val2):
                    continue
                for val3 in player3_cannot_have:
                    if(val3 != val2):
                        continue
                    for val4 in player4_cannot_have:
                        if(val4 != val3):
                            continue
                        else:
                            if val1 in solutions:
                                return
                            else:
                                solutions.append(val1)
                                print("added location 3", location)
                                time.sleep(15)
                                added_list.append([val1, solutions])
                                #fake index so it checks all
                                fill_other_players_cannot_have(val1, 20)
                                

    

def add_p_Location(_):

    global players_known 
    global choose_location
    global choose_player
    
  
    
  
    with out:
        
        clear_output(wait=True)
        
        if(check_if_others_have(choose_location.value) == False):
            if(choose_location.value != None and choose_player.value != None 
               and list_contains_value(players_known[choose_player.value], choose_location.value) == False):
                players_known[choose_player.value].append(choose_location.value)
                added_list.append([choose_location.value, players_known[choose_player.value]])
                fill_other_players_cannot_have(choose_location.value, choose_player.value)
        
        if("annex" in players_known[1]):
            with open('erroraddplocation.txt', 'w') as out_file:
                json.dump("added_anex ", out_file)
                exit()
        
        print_players()
        
        recreate_buttons()
        
def add_p_Weapon(_):

    global players_known 
    global choose_weapon
    global choose_player
    
  
  
    with out:
        
        clear_output(wait=True)
        
        if(check_if_others_have(choose_weapon.value) == False):
            if(choose_weapon.value != None and choose_player.value != None 
               and list_contains_value(players_known[choose_player.value], choose_weapon.value) == False):
                players_known[choose_player.value].append(choose_weapon.value)
                added_list.append([choose_weapon.value, players_known[choose_player.value]])
                fill_other_players_cannot_have(choose_weapon.value, choose_player.value)
        
        print_players()
        
        recreate_buttons()
        
def add_p_Person(_):

    global players_known 
    global choose_person
    global choose_player
    

  
    with out:
        
        clear_output(wait=True)
        
        if(check_if_others_have(choose_person.value) == False):
            if(choose_person.value != None and choose_player.value != None 
               and list_contains_value(players_known[choose_player.value], choose_person.value) == False):
                players_known[choose_player.value].append(choose_person.value)
                added_list.append([choose_person.value, players_known[choose_player.value]])
                fill_other_players_cannot_have(choose_person.value, choose_player.value)
        
        print_players()
        
        recreate_buttons()
def add_Sales_Location(_):

    global sales 
    global choose_location


    
    with out:
        
        clear_output(wait=True)
        if(check_if_others_have(choose_location.value) == False and choose_location.value != None):
            sales = choose_location.value
            added_list.append([choose_location.value, sales])
            #give it fake index 20 because it needs to check all players
            #giving a fake index that doesn't map to any player ensures this
            fill_other_players_cannot_have(choose_location.value, 20)
        
        print_players()
        
        recreate_buttons()
        
def add_Sales_Weapon(_):

    global sales 
    global choose_weapon


    
    with out:
        
        clear_output(wait=True)
        if(check_if_others_have(choose_weapon.value) == False and choose_weapon.value != None):
            sales = choose_weapon.value
            added_list.append([choose_weapon.value, sales])
            #give it fake index 20 because it needs to check all players
            #giving a fake index that doesn't map to any player ensures this
            fill_other_players_cannot_have(choose_weapon.value, 20)
        
        print_players()
        
        recreate_buttons()
        
def add_Sales_Person(_):

    global sales 
    global choose_person


    
    with out:
        
        clear_output(wait=True)
        
        if(check_if_others_have(choose_person.value) == False and choose_person.value != None):
            sales = choose_person.value
            added_list.append([choose_person.value, sales])
            #give it fake index 20 because it needs to check all players
            #giving a fake index that doesn't map to any player ensures this
            fill_other_players_cannot_have(choose_person.value, 20)
        
        print_players()
        
        recreate_buttons()
        
        
def deduction_check(prover_index, location, weapon, person):
    
    global players_known
    global number_players
    global sales
    
        
    prover_did_not_have_location = False
    prover_did_not_have_weapon = False
    prover_did_not_have_person = False
    
    if(sales == location):
        prover_did_not_have_location = True
    elif(sales == weapon):
        prover_did_not_have_weapon = True
    elif(sales == person):
        prover_did_not_have_person = True
    
    for index in range(number_players):
        if(index == prover_index):
            continue
        else:
            print("TESTING TESTING",players_known[index])
            if(players_known[index] != None):
                for item in players_known[index]:
                    if(item == location):
                        prover_did_not_have_location = True
                    elif(item == weapon):
                        prover_did_not_have_weapon = True
                    elif(item == person):
                        prover_did_not_have_person = True
        
    true_count = 0
    
    if(prover_did_not_have_location):
        true_count += 1
    if(prover_did_not_have_weapon):
        true_count += 1
    if(prover_did_not_have_person):
        true_count += 1
        
    if(true_count == 3):
        print_players()
        print("error impossible cannot prove and not have any")
        exit()
    elif(true_count == 2):
        
        if(prover_did_not_have_location == False):
            if location in players_known[prover_index] or location in solutions:
                return
            else:
                players_known[prover_index].append(location)
                print("added2 ", location, "to player ", prover_index + 1)
                added_list.append([location , players_known[prover_index]])
                fill_other_players_cannot_have(location, prover_index)
        elif(prover_did_not_have_weapon == False):
            if weapon in players_known[prover_index] or weapon in solutions:
                return
            else:
                players_known[prover_index].append(weapon)
                print("added2 ", weapon, "to player ", prover_index + 1)
                added_list.append([weapon, players_known[prover_index]])
                fill_other_players_cannot_have(weapon, prover_index)
        elif(prover_did_not_have_person == False):
            if person in players_known[prover_index] or person in solutions:
                return
            else:
                players_known[prover_index].append(person)
                print("added2 ", person, "to player ", prover_index + 1)
                added_list.append([person, players_known[prover_index]])
                fill_other_players_cannot_have(person, prover_index)
        
        if("annex" in players_known[1]):
            with open('errordeducion.txt', 'w') as out_file:
                json.dump("added_anex ", out_file)
                exit()
    
    
def guess_made_update(_):
    
    global choose_location
    global choose_weapon
    global choose_person
    global choose_player_guesser
    global choose_player_prover
    global players_known
    global number_players
    global solutions
    global player1_unproven_guesses
    
    with out:
            
        clear_output(wait=True)
    
        look_for_solutions_unproven_guesses()

        #mistake by user don't want to mess up whole game
        if(choose_player_guesser.value == choose_player_prover.value or choose_player_guesser.value == None):
            return

        location_guessed = choose_location.value
        weapon_guessed = choose_weapon.value
        person_guessed = choose_person.value
        
        stored_guesses_for_replay.append([location_guessed, weapon_guessed, person_guessed, choose_player_guesser.value, choose_player_prover.value])
        
        added_list.append([[location_guessed, weapon_guessed, person_guessed, choose_player_guesser.value, choose_player_prover.value], stored_guesses_for_replay])

        add_guess_to_list(location_guessed, weapon_guessed, person_guessed, choose_player_guesser.value, choose_player_prover.value)
    
    
        if(choose_player_prover.value != None):
            
            
            cross_reference_cannot_have(choose_player_prover.value, location_guessed, weapon_guessed, person_guessed)

            prover = players_known[choose_player_prover.value]

            #always assume you are player 1, player 2 is to your left, player 4 is to your right
            players_who_could_not_prove = []

            if (choose_player_guesser.value == 0 and choose_player_prover.value == 2):
                players_who_could_not_prove.append(1)

            elif (choose_player_guesser.value == 0 and choose_player_prover.value == 3):
                players_who_could_not_prove.append(1)
                players_who_could_not_prove.append(2)
            elif (choose_player_guesser.value == 1 and choose_player_prover.value == 3):
                players_who_could_not_prove.append(2)
            elif (choose_player_guesser.value == 1 and choose_player_prover.value == 0):
                players_who_could_not_prove.append(2)
                players_who_could_not_prove.append(3)
            elif (choose_player_guesser.value == 2 and choose_player_prover.value == 0):
                players_who_could_not_prove.append(3)
            elif (choose_player_guesser.value == 2 and choose_player_prover.value == 1):
                players_who_could_not_prove.append(3)
                players_who_could_not_prove.append(0)
            elif (choose_player_guesser.value == 3 and choose_player_prover.value == 1):
                players_who_could_not_prove.append(0)
                #fill in deduction code
            elif (choose_player_guesser.value == 3 and choose_player_prover.value == 2):
                players_who_could_not_prove.append(0)
                players_who_could_not_prove.append(1)
               

            for player_index in players_who_could_not_prove:
                if(list_contains_value(players_cannot_have[player_index], choose_location.value) == False):
                    players_cannot_have[player_index].append(choose_location.value)
                    added_list.append([choose_location.value, players_cannot_have[player_index]])
                if(list_contains_value(players_cannot_have[player_index], choose_weapon.value) == False):
                    players_cannot_have[player_index].append(choose_weapon.value)
                    added_list.append([choose_weapon.value, players_cannot_have[player_index]])
                if(list_contains_value(players_cannot_have[player_index], choose_person.value) == False):
                    players_cannot_have[player_index].append(choose_person.value)
                    added_list.append([choose_person.value, players_cannot_have[player_index]])
            
            
            look_through_all_previous_guesses_for_answers()

            deduction_check(choose_player_prover.value, location_guessed, weapon_guessed, person_guessed)
            
            look_through_all_previous_guesses_for_answers()
        
        #if we made the guess
        elif(choose_player_prover.value == None and choose_player_guesser.value == 0 and sales != None):
            
            
            players_who_could_not_prove = []  
            
            for i in range(number_players):
                if(i == choose_player_guesser.value):
                    continue
                else:
                    players_who_could_not_prove.append(i)

                  
                    
            #add to values other players do not have
            for player_index in players_who_could_not_prove:
                if(list_contains_value(players_cannot_have[player_index], choose_location.value) == False):
                    players_cannot_have[player_index].append(choose_location.value)
                    added_list.append([choose_location.value, players_cannot_have[player_index]])
                if(list_contains_value(players_cannot_have[player_index], choose_weapon.value) == False):
                    players_cannot_have[player_index].append(choose_weapon.value)
                    added_list.append([choose_weapon.value, players_cannot_have[player_index]])
                if(list_contains_value(players_cannot_have[player_index], choose_person.value) == False):
                    players_cannot_have[player_index].append(choose_person.value)
                    added_list.append([choose_person.value, players_cannot_have[player_index]])
            
            look_through_all_previous_guesses_for_answers()

            
            location_is_solution = True
            weapon_is_solution = True
            person_is_solution = True

            if(sales == location_guessed):
                location_is_solution = False
            elif(sales == weapon_guessed):
                weapon_is_solution = False
            elif(sales == person_guessed):
                person_is_solution = False

            for index in range(number_players):
                if location_guessed in players_known[index]:
                    location_is_solution = False
                if weapon_guessed in players_known[index]:
                    weapon_is_solution = False
                if person_guessed in players_known[index]:
                    person_is_solution = False

            if(location_is_solution):
                if location_guessed in solutions:
                    solutions = solutions
                else:
                    solutions.append(location_guessed)
                    print("added location 4", location)
                    time.sleep(15)
                    added_list.append([location_guessed, solutions])
                    #fake index 20 to check all players
                    fill_other_players_cannot_have(location_guessed, 20)
            if(weapon_is_solution):
                if weapon_guessed in solutions:
                    solutions = solutions
                else:
                    solutions.append(weapon_guessed)
                    added_list.append([weapon_guessed, solutions])
                    #fake index 20 to check all players
                    fill_other_players_cannot_have(weapon_guessed, 20)
            if(person_is_solution):
                if person_guessed in solutions:
                    solutions = solutions
                else:
                    solutions.append(person_guessed)
                    added_list.append([person_guessed, solutions])
                    #fake index 20 to check all players
                    fill_other_players_cannot_have(person_guessed, 20)
            
            look_through_all_previous_guesses_for_answers()
        #if someone else made a guess that couldn't be proved
        elif(choose_player_prover.value == None and choose_player_guesser.value != 0 and sales != None):

            
            players_who_could_not_prove = []     
            
            for i in range(number_players):
                if(i == choose_player_guesser.value):
                    continue
                else:
                    players_who_could_not_prove.append(i)
   
            
            #add to values other players do not have
            for player_index in players_who_could_not_prove:
                if(list_contains_value(players_cannot_have[player_index], choose_location.value) == False):
                    players_cannot_have[player_index].append(choose_location.value)
                    added_list.append([choose_location.value, players_cannot_have[player_index]])
                if(list_contains_value(players_cannot_have[player_index], choose_weapon.value) == False):
                    players_cannot_have[player_index].append(choose_weapon.value)
                    added_list.append([choose_weapon.value, players_cannot_have[player_index]])
                if(list_contains_value(players_cannot_have[player_index], choose_person.value) == False):
                    players_cannot_have[player_index].append(choose_person.value)
                    added_list.append([choose_person.value, players_cannot_have[player_index]])
            
            look_through_all_previous_guesses_for_answers()
                    
            check_for_definite_solutions()
            
            look_through_all_previous_guesses_for_answers()
            
        #if player 1 makes a guess but sales isn't yet known
        elif(choose_player_prover.value == None and choose_player_guesser.value == 0 and sales == None):
            if [location_guessed, weapon_guessed, person_guessed] in player1_unproven_guesses:
                player1_unproven_guesses = player1_unproven_guesses
            else: 
                player1_unproven_guesses.append([location_guessed, weapon_guessed, person_guessed])
                look_for_solutions_unproven_guesses()
            
            look_through_all_previous_guesses_for_answers()
                
        print_players()

        if(len(solutions) == 3):
            print("ALL SOLUTIONS FOUND!!!!")
            
def guess_made_manual(location_guessed, weapon_guessed, person_guessed, choose_player_guesser, choose_player_prover):
    
    global choose_location
    global choose_weapon
    global choose_person
    global players_known
    global number_players
    global solutions
    global player1_unproven_guesses
    
    with out:
            
        clear_output(wait=True)
    
        look_for_solutions_unproven_guesses()

        #mistake by user don't want to mess up whole game
        if(choose_player_guesser == choose_player_prover or choose_player_guesser == None):
            return
        

        add_guess_to_list(location_guessed, weapon_guessed, person_guessed, choose_player_guesser, choose_player_prover)
    

    
    
        if(choose_player_prover != None):
            
            
            cross_reference_cannot_have(choose_player_prover, location_guessed, weapon_guessed, person_guessed)

            prover = players_known[choose_player_prover]

            #always assume you are player 1, player 2 is to your left, player 4 is to your right
            players_who_could_not_prove = []

            if (choose_player_guesser == 0 and choose_player_prover == 2):
                players_who_could_not_prove.append(1)

            elif (choose_player_guesser == 0 and choose_player_prover == 3):
                players_who_could_not_prove.append(1)
                players_who_could_not_prove.append(2)
            elif (choose_player_guesser == 1 and choose_player_prover == 3):
                players_who_could_not_prove.append(2)
            elif (choose_player_guesser == 1 and choose_player_prover == 0):
                players_who_could_not_prove.append(2)
                players_who_could_not_prove.append(3)
            elif (choose_player_guesser == 2 and choose_player_prover == 0):
                players_who_could_not_prove.append(3)
            elif (choose_player_guesser == 2 and choose_player_prover == 1):
                players_who_could_not_prove.append(3)
                players_who_could_not_prove.append(0)
            elif (choose_player_guesser == 3 and choose_player_prover == 1):
                players_who_could_not_prove.append(0)
                #fill in deduction code
            elif (choose_player_guesser == 3 and choose_player_prover == 2):
                players_who_could_not_prove.append(0)
                players_who_could_not_prove.append(1)
               

            for player_index in players_who_could_not_prove:
                if(list_contains_value(players_cannot_have[player_index], location_guessed) == False):
                    players_cannot_have[player_index].append(location_guessed)
                    added_list.append([location_guessed, players_cannot_have[player_index]])
                if(list_contains_value(players_cannot_have[player_index], weapon_guessed) == False):
                    players_cannot_have[player_index].append(weapon_guessed)
                    added_list.append([weapon_guessed, players_cannot_have[player_index]])
                if(list_contains_value(players_cannot_have[player_index], person_guessed) == False):
                    players_cannot_have[player_index].append(person_guessed)
                    added_list.append([person_guessed, players_cannot_have[player_index]])
            
            
            look_through_all_previous_guesses_for_answers()

            deduction_check(choose_player_prover, location_guessed, weapon_guessed, person_guessed)
            
            look_through_all_previous_guesses_for_answers()
        
        #if we made the guess
        elif(choose_player_prover == None and choose_player_guesser == 0 and sales != None):
            
            
            players_who_could_not_prove = []  
            
            for i in range(number_players):
                if(i == choose_player_guesser):
                    continue
                else:
                    players_who_could_not_prove.append(i)

                  
                    
            #add to values other players do not have
            for player_index in players_who_could_not_prove:
                if(list_contains_value(players_cannot_have[player_index], location_guessed) == False):
                    players_cannot_have[player_index].append(location_guessed)
                    added_list.append([location_guessed, players_cannot_have[player_index]])
                if(list_contains_value(players_cannot_have[player_index], weapon_guessed) == False):
                    players_cannot_have[player_index].append(weapon_guessed)
                    added_list.append([weapon_guessed, players_cannot_have[player_index]])
                if(list_contains_value(players_cannot_have[player_index], person_guessed) == False):
                    players_cannot_have[player_index].append(person_guessed)
                    added_list.append([person_guessed, players_cannot_have[player_index]])
            
            look_through_all_previous_guesses_for_answers()

            
            location_is_solution = True
            weapon_is_solution = True
            person_is_solution = True

            if(sales == location_guessed):
                location_is_solution = False
            elif(sales == weapon_guessed):
                weapon_is_solution = False
            elif(sales == person_guessed):
                person_is_solution = False

            for index in range(number_players):
                if location_guessed in players_known[index]:
                    location_is_solution = False
                if weapon_guessed in players_known[index]:
                    weapon_is_solution = False
                if person_guessed in players_known[index]:
                    person_is_solution = False

            if(location_is_solution):
                if location_guessed in solutions:
                    solutions = solutions
                else:
                    solutions.append(location_guessed)
                    print("added location 5", location)
                    time.sleep(15)
                    added_list.append([location_guessed, solutions])
                    #fake index 20 to check all players
                    fill_other_players_cannot_have(location_guessed, 20)
            if(weapon_is_solution):
                if weapon_guessed in solutions:
                    solutions = solutions
                else:
                    solutions.append(weapon_guessed)
                    added_list.append([weapon_guessed, solutions])
                    #fake index 20 to check all players
                    fill_other_players_cannot_have(weapon_guessed, 20)
            if(person_is_solution):
                if person_guessed in solutions:
                    solutions = solutions
                else:
                    solutions.append(person_guessed)
                    added_list.append([person_guessed, solutions])
                    #fake index 20 to check all players
                    fill_other_players_cannot_have(person_guessed, 20)
            
            look_through_all_previous_guesses_for_answers()
        #if someone else made a guess that couldn't be proved
        elif(choose_player_prover == None and choose_player_guesser != 0 and sales != None):

            
            players_who_could_not_prove = []     
            
            for i in range(number_players):
                if(i == choose_player_guesser):
                    continue
                else:
                    players_who_could_not_prove.append(i)
   
            
            #add to values other players do not have
            for player_index in players_who_could_not_prove:
                if(list_contains_value(players_cannot_have[player_index], location_guessed) == False):
                    players_cannot_have[player_index].append(location_guessed)
                    added_list.append([location_guessed, players_cannot_have[player_index]])
                if(list_contains_value(players_cannot_have[player_index], weapon_guessed) == False):
                    players_cannot_have[player_index].append(weapon_guessed)
                    added_list.append([weapon_guessed, players_cannot_have[player_index]])
                if(list_contains_value(players_cannot_have[player_index], person_guessed) == False):
                    players_cannot_have[player_index].append(person_guessed)
                    added_list.append([person_guessed, players_cannot_have[player_index]])
            
            look_through_all_previous_guesses_for_answers()
                    
            check_for_definite_solutions()
            
            look_through_all_previous_guesses_for_answers()
            
        #if player 1 makes a guess but sales isn't yet known
        elif(choose_player_prover == None and choose_player_guesser == 0 and sales == None):
            if [location_guessed, weapon_guessed, person_guessed] in player1_unproven_guesses:
                player1_unproven_guesses = player1_unproven_guesses
            else: 
                player1_unproven_guesses.append([location_guessed, weapon_guessed, person_guessed])
                look_for_solutions_unproven_guesses()
            
            look_through_all_previous_guesses_for_answers()
                
        print_players()

        if(len(solutions) == 3):
            print("ALL SOLUTIONS FOUND!!!!")

def store_guesses(_):
    
    global stored_guesses_for_replay
    global players_known
    global players_cannot_have
    global number_players
    global sales
    global solutions
    global all_guesses_list
    global player1_unproven_guesses
    
    
#     CustomObjectListsToJSON.to_json()
    

    CustomObjectListsToJSON.to_json(copy.deepcopy(stored_guesses_for_replay), 'guesses', [locations, weapons, people, int, list])
    
    for i in range(number_players):
        CustomObjectListsToJSON.to_json(copy.deepcopy(players_known[i]), 'player'+str(i+1)+'_known', [locations, weapons, people, int, list])
            
    for i in range(number_players):
        CustomObjectListsToJSON.to_json(copy.deepcopy(players_cannot_have[i]), 'player'+str(i+1)+'_cannot_have', [locations, weapons, people, int, list])
    
    if(sales != None):
        CustomObjectListsToJSON.to_json([sales], 'sales', [locations, weapons, people, int, list])
    else:
        CustomObjectListsToJSON.to_json([None], 'sales', [locations, weapons, people, int, list])
        
        
    CustomObjectListsToJSON.to_json(copy.deepcopy(solutions), 'solutions', [locations, weapons, people, int, list])
    CustomObjectListsToJSON.to_json(copy.deepcopy(all_guesses_list), 'all_guesses_list', [locations, weapons, people, int, list])
    CustomObjectListsToJSON.to_json(copy.deepcopy(player1_unproven_guesses), 'player1_unproven_guesses', [locations, weapons, people, int, list])
    
        
def restore_guesses_and_play(_):
    global stored_guesses_for_replay
    global players_known
    global player1_known
    global player2_known
    global player3_known
    global player4_known
    global player1_cannot_have
    global player2_cannot_have
    global player3_cannot_have
    global player4_cannot_have
    global sales
    global solutions
    global all_guesses_list
    global player1_unproven_guesses
    
    

    stored_guesses_for_replay = CustomObjectListsToJSON.from_json('guesses', [locations, weapons, people, int, list])
        
    for i in range(number_players):
        players_known[i] = CustomObjectListsToJSON.from_json('player'+str(i+1)+'_known', [locations, weapons, people, int, list])
        players_known[i] = int_to_enumint(players_known[i])
        if(i == 0):
            player1_known = players_known[i]
        elif(i == 1):
            player2_known = players_known[i]
        elif(i == 2):
            player3_known = players_known[i]
        elif(i == 3):
            player4_known = players_known[i]
    
    for i in range(number_players):
        players_cannot_have[i] = CustomObjectListsToJSON.from_json('player'+str(i+1)+'_cannot_have', [locations, weapons, people, int, list])
        players_cannot_have[i] = int_to_enumint(players_cannot_have[i])
        if(i == 0):
            player1_cannot_have = players_cannot_have[i]
        elif(i == 1):
            player2_cannot_have = players_cannot_have[i]
        elif(i == 2):
            player3_cannot_have = players_cannot_have[i]
        elif(i == 3):
            player4_cannot_have = players_cannot_have[i]
      
    sales = players_known[i] = int_to_enumint(CustomObjectListsToJSON.from_json('sales', [locations, weapons, people, int, list]))
    
    if(len(sales) == 0):
        sales = None
    else:
        temp_dat = sales[0]
        sales = temp_dat
        
    solutions = int_to_enumint(CustomObjectListsToJSON.from_json('solutions', [locations, weapons, people, int, list]))
              

    all_guesses_list = CustomObjectListsToJSON.from_json('all_guesses_list', [locations, weapons, people, int, list])
    for i in range(len(all_guesses_list)):
          all_guesses_list[i][0:3] = int_to_enumint(all_guesses_list[i][0:3])
    
    
    player1_unproven_guesses = CustomObjectListsToJSON.from_json('player1_unproven_guesses', [locations, weapons, people, int, list])
    for i in range(len(player1_unproven_guesses)):
        player1_unproven_guesses[i] = int_to_enumint(player1_unproven_guesses[i])
                
    
    print_players()
        
#     for guess in stored_guesses_for_replay:
#         guess_made_manual(locations(guess[0]),weapons(guess[1]),people(guess[2]),guess[3],guess[4],)

def int_to_enumint(list_ints):
    
    if(len(list_ints) == 1 and list_ints[0] == None):
        return [None]
    
    for i in range(len(list_ints)):
        if(locations.has_value(list_ints[i])):
            list_ints[i] = locations(list_ints[i])
        elif(weapons.has_value(list_ints[i])):
            list_ints[i] = weapons(list_ints[i])
        elif(people.has_value(list_ints[i])):
            list_ints[i] = people(list_ints[i])
    
    return list_ints
            
    
   
    
def recreate_buttons():
    add_player_hand_location.on_click(add_p_Location)
    add_player_hand_weapon.on_click(add_p_Weapon)
    add_player_hand_person.on_click(add_p_Person)
    fill_player1_cannot_have.on_click(fill_p1_cannot_have)
    add_location_sales.on_click(add_Sales_Location)
    add_weapon_sales.on_click(add_Sales_Weapon)
    add_person_sales.on_click(add_Sales_Person)
    add_guess.on_click(guess_made_update)
    undo_previous.on_click(remove_most_recently_added)
    refresh.on_click(print_players_refresh)
    store.on_click(store_guesses)
    restore.on_click(restore_guesses_and_play)


    widgets.VBox([add_player_hand_location, add_player_hand_weapon, add_player_hand_person, fill_player1_cannot_have, choose_player, 
                  choose_player_guesser, choose_player_prover, choose_location, choose_weapon, choose_person, add_guess, add_location_sales,
                  add_weapon_sales, add_person_sales, undo_previous, refresh, out, store, restore])
    



add_player_hand_location.on_click(add_p_Location)
add_player_hand_weapon.on_click(add_p_Weapon)
add_player_hand_person.on_click(add_p_Person)
fill_player1_cannot_have.on_click(fill_p1_cannot_have)
add_location_sales.on_click(add_Sales_Location)
add_weapon_sales.on_click(add_Sales_Weapon)
add_person_sales.on_click(add_Sales_Person)
add_guess.on_click(guess_made_update)
undo_previous.on_click(remove_most_recently_added)
refresh.on_click(print_players_refresh)
store.on_click(store_guesses)
restore.on_click(restore_guesses_and_play)


widgets.VBox([add_player_hand_location, add_player_hand_weapon, add_player_hand_person, fill_player1_cannot_have, choose_player, 
              choose_player_guesser, choose_player_prover, choose_location, choose_weapon, choose_person, add_guess, add_location_sales,
              add_weapon_sales, add_person_sales, undo_previous, refresh, out, store, restore])

Populating the interactive namespace from numpy and matplotlib


C:\Users\laika\anaconda3\lib\site-packages\IPython\core\magics\pylab.py:159: UserWarning: pylab import has clobbered these variables: ['copy']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [34]:
# for i in range(10, 20):
#     print("i", i)
#     for j in range(15, 20):
#         if(i == j):
#             breakall
#         print("j",j)

# print(player2_known)
# print(player2_cannot_have)
# print(all_guesses_list)

for guess in all_guesses_list:
    print(guess, "\n")

[<locations.Reception: 7>, <weapons.Dundy_Trophy: 13>, <people.Jim: 18>, 2, 0] 

[<locations.Parking_Lot: 4>, <weapons.Worlds_Best_Boss_Mug: 11>, <people.Pam: 23>, 3, 1] 

[<locations.Annex: 2>, <weapons.Nunchunks: 15>, <people.Dwight: 20>, 0, 1] 

[<locations.Accounting: 8>, <weapons.Dundy_Trophy: 13>, <people.Jim: 18>, 2, 3] 

[<locations.Annex: 2>, <weapons.Worlds_Best_Boss_Mug: 11>, <people.Dwight: 20>, 3, 0] 

[<locations.Parking_Lot: 4>, <weapons.Mr_A_Knife: 10>, <people.Pam: 23>, 0, 1] 

[<locations.Warehouse: 5>, <weapons.Dundy_Trophy: 13>, <people.Jim: 18>, 2, 1] 

[<locations.Annex: 2>, <weapons.Ream_Of_Paper: 16>, <people.Pam: 23>, 0, None] 

[<locations.Accounting: 8>, <weapons.Dundy_Trophy: 13>, <people.Stanley: 19>, 1, 2] 

[<locations.Break_Room: 0>, <weapons.Ream_Of_Paper: 16>, <people.Stanley: 19>, 2, 3] 

[<locations.Conference_Room: 1>, <weapons.Worlds_Best_Boss_Mug: 11>, <people.Dwight: 20>, 3, 1] 

[<locations.Accounting: 8>, <weapons.Bacon_Grill: 17>, <people.Stan

In [ ]:
print(str(locations.Break_Room.name))

In [ ]:
x = 3
y = x 

x = 5

print(y)

In [ ]:
for i in locations:
    print(i)

In [ ]:
for i in range(1, 4):
    print(i)

In [ ]:
my_list = [5]

my_bool = 5 in my_list

print(my_bool)

In [ ]:
bool1 = False
bool2 = True
bool3 = False

if(not bool1 and bool2 and not bool3):
    print("HI")

In [ ]:

ints_set1 = [1, 2, 3, 4, 5]
ints_set2 = [6, 7, 8, 9, 10]
ints_set3 = [11, 12, 13, 14, 15]

total_set = [ints_set1,ints_set2,ints_set3,]

extra = 18

item = 17

in_any_set = set(item in my_set for my_set in total_set)

in_any_set.add(extra != item)

print(in_any_set)

print(True in in_any_set)

print(len(in_any_set))


In [ ]:
print(in_any_set)

In [ ]:
data = [[<locations.Reception: 7>, <weapons.Dundy_Trophy: 13>, <people.Jim: 18>, 2, 0], [<locations.Parking_Lot: 4>, <weapons.Worlds_Best_Boss_Mug: 11>, <people.Pam: 23>, 3, 1], [<locations.Annex: 2>, <weapons.Nunchunks: 15>, <people.Dwight: 20>, 0, 1]]

In [ ]:
data = [locations.Michaels_Office, weapons.Poisoned_Pretzel, people.Andy, 2, 1]

In [ ]:
for datitem in data:
    print(type(datitem))

In [ ]:
print(player1_known)

In [ ]:
print(locations(1))

In [ ]:
#idea... add to int enum class


def __str__(self):
        return str(self.value)
    
print(locations(30))

In [ ]:
print(weapons.has_value(4))

In [ ]:
my_list2d = [[1, 2, 3, 4, 5], [1, 2, 3, 4]]

my_list1d = [1, 2, 3, 4]

print(type(my_list1d[0]))
print(len(np.shape(my_list1d)))
print(np.shape(my_list1d))
print(type(my_list1d) == list)
# print(type(np.shape(my_list)))
print(type(None))


my_list_lone = [1]

my_listtypes = [locations, int]

print(my_listtypes.index(int))

my_list_lone[0] = my_listtypes[0](my_list_lone[0])

print(my_list_lone)

In [ ]:
import numpy as np
import json 
from enum import IntEnum



def list_with_type_toJSON(my_list_i, types):
    print('called main')
    current_list = my_list_i
    for item in current_list:
        if(type(item) == list):
#             print("current item", item)
#             print('0')
            list_with_type_toJSON(item, types)
        else:
            for i in range(len(current_list)):
#                 print("iteminner", current_list[i])
#                 print('1')
                temp_dat = current_list[i]
                if(temp_dat != None):
                    temp_dat_type_index = types.index(type(current_list[i]))
                    current_list[i] = [temp_dat, temp_dat_type_index]
                else:
                    current_list[i] = [temp_dat, None]
            return my_list_i
                
    return my_list_i


def JSON_to_list_with_type(my_list_i, types):
    print('called main')
    print('current list')
    current_list = my_list_i
    for item in current_list:
        if(type(item[0]) == list):
            print('0')
            print(item)
            JSON_to_list_with_type(item, types)
        else:
            for i in range(len(current_list)):
                print('1')
                print(current_list)
                temp_dat = current_list[i][0]
                if(temp_dat != None):
                    temp_dat_type_index = current_list[i][1]
                    current_list[i] = types[temp_dat_type_index](temp_dat)
                else:
                    current_list[i] = None
            return my_list_i
                
    return my_list_i




#list_to_save = a list of different type items to save to JSON
#filename = name of the file
#types = a list of all different types contained in the list
def list_of_many_types_to_JSON(list_to_save, filename, types):

    
    #list to return
    json_list = []
    
    
#     print(list_to_save)
    
    json_list = list_with_type_toJSON(list_to_save, types)
    
#     print('post good', json_list)

        
    with open(filename+'.txt', 'w') as out_file:
        json.dump(json_list, out_file)

    return 1
    
# print(list_of_many_types_to_JSON([locations.Accounting, people.Andy], "unit_test_file", [locations, people]))

def JSON_to_list_of_many_types(filename, types):
    
    loaded_json = []
    
    print(filename)
    print(type(filename))
    
    with open(filename+'.txt', 'r') as in_file:
        loaded_json = json.load(in_file)
        
    print("loaded_json", loaded_json)

    return_list = []
    
    return_list = JSON_to_list_with_type(loaded_json, types)
    
    return return_list
            
            

In [ ]:
import unittest




class TestNotebook(unittest.TestCase):
    
    def test_list_to_json(self):
        self.assertEqual(list_of_many_types_to_JSON([locations.Accounting, people.Andy], "unit_test_file", [locations, people]), 1)
    def test_json_to_list(self):
        self.assertEqual(JSON_to_list_of_many_types("unit_test_file", [locations, people]), [locations.Accounting, people.Andy])
        

unittest.main(argv=[''], verbosity=2, exit=False)

In [ ]:
print(len(0))

In [ ]:
my_list = [[[1, 2, 3, 4], [1, 2, 3, 4, 5]], [[1, 2, 3, 4], [1, 2, 3, 4, 5]]]

# for alist in my_list:
#     for i in range(len(alist)):
#         alist[i] = 9
    
# print(my_list)

In [ ]:
def new_list(my_list):
    current_list = my_list
    for item in current_list:
        if(type(item) == list):
            new_list(item)
        else:
            for i in range(len(current_list)):
                current_list[i] = 9
                
    return my_list

In [ ]:
print(my_list)
print(new_list(my_list))

In [ ]:
print(all_guesses_list)

In [ ]:


list_of_many_types_to_JSON(copy.deepcopy(all_guesses_list), "test123", [locations, weapons, people, int, None, list])
retrieved = JSON_to_list_of_many_types("test123", [locations, weapons, people, int, None, list])

print(retrieved)

In [3]:
print([None][0])

None


In [36]:
all_locations = []

for loc in locations:
    all_locations.append(loc)

print(all_locations)

[<locations.Break_Room: 0>, <locations.Conference_Room: 1>, <locations.Annex: 2>, <locations.Michaels_Office: 3>, <locations.Parking_Lot: 4>, <locations.Warehouse: 5>, <locations.Kitchen: 6>, <locations.Reception: 7>, <locations.Accounting: 8>]
